In [2]:
import numpy as np
import pandas as pd


## `calculate_entropy` Function explained in detail
computes the entropy of a set of labels.
1. **Input**:
   - It takes a set of labels (`y`) as input.

2. **Unique Labels and Counts**:
   - It identifies the unique labels present in the input along with their respective counts using `np.unique(y, return_counts=True)`.

3. **Probability Calculation**:
   - For each unique label, it calculates the probability by dividing the count of each label by the total number of labels in the input.

4. **Entropy Calculation**:
   - Utilizing the probabilities of the labels, it computes the entropy using the standard entropy formula: \( -\sum_{i} p_i \log_2(p_i) \), where \(p_i\) represents the probability of each unique label.

5. **Return**:
   - The function returns the computed entropy value.

This function is crucial in the decision tree algorithm as it's used to determine the impurity or disorder in a set of labels, a key factor in deciding how to split the data effectively at each node of the tree. The lower the entropy, the more homogenous the set of labels, making it a potential stopping point in the tree's growth.

In [ ]:

def calculate_entropy(y):
    """
    Calculate the entropy of a set of labels.

    Args:
        y (array-like): Target variable labels.

    Returns:
        float: Entropy value.
    """
    unique_labels, label_counts = np.unique(y, return_counts=True)
    probabilities = label_counts / len(y)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy





:
## find_best_split Function explained in detail
1. **Initialization**:
   - `best_feature`, `best_value`, and `best_information_gain` are initialized to None and -1 respectively. These variables will hold the best split found and the information gain achieved.

2. **Calculate Current Entropy**:
   - The current entropy for the provided subset of data (`node_indices`) is calculated using the `calculate_entropy` function.

3. **Iterating through Features**:
   - It loops through each feature in the dataset.

4. **Iterating through Unique Values of the Feature**:
   - For each feature, it identifies unique values in that feature within the provided subset.

5. **Splitting the Data**:
   - For each value of the feature, it splits the dataset into two groups - left and right - based on whether the feature value is less than or greater than the selected value.

6. **Entropy Calculation**:
   - It calculates the entropy for both the left and right subsets.

7. **Information Gain Calculation**:
   - Using the entropy of the subsets, it calculates the information gain by considering how much each split reduces the entropy in comparison to the starting entropy of the whole dataset.

8. **Update Best Split**:
   - If the information gain from the current split is higher than the previous best information gain, it updates the `best_information_gain`, `best_feature`, and `best_value`.

9. **Return Best Split**:
   - Finally, it returns the best feature and value found, which represent the optimal way to split the data to maximize information gain.

This function essentially searches for the best feature and value combination to split the data, aiming to maximize the information gain, a crucial step in building a decision tree.


In [4]:

def find_best_split(X, y, node_indices):
    """
    Find the best feature and value to split the dataset based on information gain.

    Args:
        X (ndarray): Data matrix of shape (n_samples, n_features).
        y (array-like): Target variable labels.
        node_indices (ndarray): List containing the active indices.

    Returns:
        tuple: Best feature index, best split value.
    """
    best_feature = None
    best_value = None
    best_information_gain = -1

    current_entropy = calculate_entropy(y[node_indices])

    for feature_index in range(X.shape[1]):
        unique_values = np.unique(X[node_indices, feature_index])

        for value in unique_values:
            left_indices = node_indices[X[node_indices, feature_index] <= value]
            right_indices = node_indices[X[node_indices, feature_index] > value]

            if len(left_indices) == 0 or len(right_indices) == 0:
                continue

            left_entropy = calculate_entropy(y[left_indices])
            right_entropy = calculate_entropy(y[right_indices])

            information_gain = current_entropy - (
                    len(left_indices) / len(node_indices) * left_entropy +
                    len(right_indices) / len(node_indices) * right_entropy
            )

            if information_gain > best_information_gain:
                best_information_gain = information_gain
                best_feature = feature_index
                best_value = value

    return best_feature, best_value



## build_tree_recursive Function explained in detail

1. **Input**:
   - It takes several parameters:
     - `X`: Data matrix of shape (n_samples, n_features).
     - `y`: List or array containing the target variable.
     - `node_indices`: List containing the active indices, i.e., the samples considered in this step.
     - `branch_name`: Name of the branch in the tree (e.g., 'Root', 'Left', 'Right').
     - `max_depth`: Maximum depth of the resulting tree.
     - `current_depth`: The current depth in the recursive process.

2. **Printing Node Information**:
   - It prints information about the current node such as the branch name, current depth, number of samples at the node, and the distribution of classes in those samples.

3. **Stopping Conditions**:
   - It checks stopping conditions:
     - If the current depth equals the maximum depth or if there's only one unique label in the node, it returns, indicating the end of that branch.
     - This establishes a limit on the depth of the tree or if a pure node (only one class) is reached.

4. **Finding Best Split**:
   - It calls the `find_best_split` function to identify the best feature and value to split the data further.

5. **Splitting Data**:
   - If a valid split is found, it divides the data into left and right subsets based on the best feature and value.

6. **Recursive Calls**:
   - It recursively calls itself for the left and right subsets, incrementing the depth for each call until a stopping condition is met.

This function essentially constructs the decision tree structure by identifying the best splits and branching into subsequent nodes, building the tree in a recursive manner until the stopping conditions are met. The printed information aids in visualizing the structure of the tree and understanding the splits being made at each level.

In [11]:

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that splits the dataset into 2 subgroups at each node.
    This function just prints the tree.

    Args:
        X (ndarray):            Data matrix of shape (n_samples, n_features).
        y (array-like):         List or ndarray with n_samples containing the target variable.
        node_indices (ndarray): List containing the active indices, i.e., the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right'].
        max_depth (int):        Max depth of the resulting tree.
        current_depth (int):    Current depth. Parameter used during recursive call.
    """
    unique_labels = np.unique(y[node_indices])
    print(f"{branch_name} Depth: {current_depth}, Samples: {len(node_indices)}, Class Distribution: {dict(zip(unique_labels, np.bincount(y[node_indices])))}")

    if current_depth == max_depth or len(unique_labels) == 1:
        return

    best_feature, best_value = find_best_split(X, y, node_indices)

    if best_feature is not None:
        print(f"{branch_name} Splitting at Feature {best_feature} with Value {best_value}")
        left_indices = node_indices[X[node_indices, best_feature] <= best_value]
        right_indices = node_indices[X[node_indices, best_feature] > best_value]

        build_tree_recursive(X, y, left_indices, branch_name="Left", max_depth=max_depth, current_depth=current_depth + 1)
        build_tree_recursive(X, y, right_indices, branch_name="Right", max_depth=max_depth, current_depth=current_depth + 1)



You will start by loading the dataset for this task. The dataset you have collected is as follows:

| glasses | Tall | Long hair | man/woman(0/1) |
|:-------:|:----:|:---------:|:--------------:|
|    1    |  1   |     1     |       1        |
|    0    |  0   |     1     |       1        |
|    1    |  0   |     0     |       0        |
|    1    |  0   |     0     |       0        |
|    0    |  0   |     1     |       1        |
|    1    |  1   |     1     |       0        |
|    1    |  0   |     0     |       0        |
|    0    |  1   |     1     |       1        |
|    0    |  1   |     1     |       1        |
|    0    |  1   |     1     |       0        |

In [15]:

# Create a DataFrame from the provided data
data = {
    'glasses': [1, 0, 1, 1, 0, 1, 1, 0, 0, 0],
    'Tall': [1, 0, 0, 0, 0, 1, 0, 1, 1, 1],
    'Long_hair': [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
    'man_woman': [1, 1, 0, 0, 1, 0, 0, 1, 1, 0]
}

df = pd.DataFrame(data)

# Display the DataFrame
print("Original Dataset:")
print(df)
print("\n")

# Convert DataFrame to NumPy arrays
X = df[['glasses', 'Tall', 'Long_hair']].values
y = df['man_woman'].values

# Run the build_tree_recursive function on the dataset
build_tree_recursive(X, y, node_indices=np.arange(len(y)), branch_name="Root", max_depth=3, current_depth=0)


Original Dataset:
   glasses  Tall  Long_hair  man_woman
0        1     1          1          1
1        0     0          1          1
2        1     0          0          0
3        1     0          0          0
4        0     0          1          1
5        1     1          1          0
6        1     0          0          0
7        0     1          1          1
8        0     1          1          1
9        0     1          1          0


Root Depth: 0, Samples: 10, Class Distribution: {0: 5, 1: 5}
Root Splitting at Feature 2 with Value 0
Left Depth: 1, Samples: 3, Class Distribution: {0: 3}
Right Depth: 1, Samples: 7, Class Distribution: {0: 2, 1: 5}


KeyboardInterrupt: 